# Setup Geral

Se estiver executando este exercício no Google Colab, execute as próximas duas células. 

Caso esteja executando localmente, não é necessário executar mas certifique-se de que o **pyspark** está instalado e configurado em sua máquina.

In [146]:
%%bash

# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (93.8 kB/s)
Reading package li

In [147]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Teste

O teste a ser realizado é composto de 3 partes:

- um exercício de programação em Python
- alguns exercícios de programação em SQL
- alguns exercícios de programação com PySpark

Você pode escolher qual das partes do exercício vai fazer primeiro. Todo o exercício deve ser completado no período de 90 minutos.

# Python

In [148]:
# SETUP
nomes_alunos = [
    ('Maria', 1),
    ('João', 2),
    ('Pedro', 3),
    ('Gabriella', 4),
    ('Giovana', 5),
    ('Arthur', 6)
]

notas_alunos = {
    1: 9.5,
    2: 5.1,
    3: 8.7,
    4: 7.1,
    5: 4.8,
    6: 6.3
}

Implemente uma função que recebe uma lista de nomes de alunos, um dicionário de notas dos mesmo, sendo que essas estruturas se relacionam por um ID.

A função deve retornar em ordem alfabética, o nome dos alunos que obtiveram notas maior ou igual de uma nota de corte informada.

In [149]:
def filtra_alunos_acima_corte(alunos, notas, nota_corte):
  x = 0
  corte = []
  while len(alunos) > x:
    nome = alunos[x][0]
    nota = notas[alunos[x][1]]
    if nota >= nota_corte:
      corte.append((nome,nota))
    x += 1
  return sorted(corte)
  
filtra_alunos_acima_corte(nomes_alunos, notas_alunos, 6)

[('Arthur', 6.3), ('Gabriella', 7.1), ('Maria', 9.5), ('Pedro', 8.7)]

# SQL

**Setup**


In [150]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

mkdir: cannot create directory ‘bases_teste’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0   5439      0 --:--:-- --:--:-- --:--:--  5439
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  2215k      0 --:--:-- --:--:-- --:--:-- 2206k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0  1056k      0 --:--:-- --:--:-- --:--:-- 1056k


In [151]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [152]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

**Função para execução de queries**

In [153]:
def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

Para executar alguma consulta, basta colocar seu código sql dentro da função q como no exemplo abaixo:

```python
q("""
    SELECT *
    FROM usuarios
""")
```

e o resultado será exibido na tela.

---

Nesta parte da atividade, você vai trabalhar com três tabelas:

- produtos
- usuarios
- vendas

Use-as para responder às perguntas a seguir.

1) Qual foi a quantidade de vendas nos estados de Minas Gerais e São Paulo para cada ano e mês?

In [154]:
q("""
    SELECT count(y.cod_produto) qtd_vendas, 
      sum(y.quantidade) qtd_itens, 
      x.estado, 
      month(y.data_compra) mes, 
      year(y.data_compra) ano
    FROM usuarios x 
    LEFT JOIN vendas y ON x.cod_usuario = y.cod_usuario
    WHERE x.estado IN ('Minas Gerais', 'São Paulo') AND y.quantidade IS NOT NULL
    GROUP BY x.estado, ano, mes
    ORDER BY ano, mes;
""")

+----------+---------+------------+---+----+
|qtd_vendas|qtd_itens|estado      |mes|ano |
+----------+---------+------------+---+----+
|1         |10       |Minas Gerais|6  |2018|
|1         |9        |São Paulo   |7  |2018|
|1         |16       |Minas Gerais|8  |2018|
|1         |12       |São Paulo   |10 |2018|
|1         |20       |Minas Gerais|10 |2018|
|1         |10       |Minas Gerais|11 |2018|
|2         |39       |Minas Gerais|12 |2018|
|2         |20       |Minas Gerais|1  |2019|
|1         |2        |Minas Gerais|2  |2019|
|2         |26       |Minas Gerais|4  |2019|
|1         |11       |Minas Gerais|5  |2019|
|1         |10       |São Paulo   |6  |2019|
|1         |1        |Minas Gerais|6  |2019|
|2         |14       |Minas Gerais|7  |2019|
|1         |17       |São Paulo   |8  |2019|
|1         |15       |Minas Gerais|8  |2019|
|2         |27       |Minas Gerais|9  |2019|
|1         |3        |Minas Gerais|10 |2019|
|1         |1        |Minas Gerais|11 |2019|
|3        

2) Quais são os usuários por Estado que mais compraram em todo o período analisado e qual foi o número de compras realizadas, a quantidade total de itens comprados e valor total pago por usuário?

In [155]:
q("""
    SELECT x.cod_usuario, 
    count(y.quantidade) qtd_vendas, 
    sum(y.quantidade) qtd_itens,
    cast(sum(valor) AS decimal(10,2)) valor_total, 
    x.estado
    FROM usuarios x 
    left join vendas y
    on x.cod_usuario =  y.cod_usuario
    where y.quantidade is not null
    group by x.cod_usuario, x.estado
    order by qtd_vendas desc;
""")

+-----------+----------+---------+-----------+-------------------+
|cod_usuario|qtd_vendas|qtd_itens|valor_total|estado             |
+-----------+----------+---------+-----------+-------------------+
|2811       |13        |117      |184101.89  |Maranhão           |
|3403       |10        |94       |136347.01  |Rio Grande do Norte|
|2843       |10        |114      |137228.01  |Paraná             |
|2340       |9         |112      |165197.30  |Mato Grosso        |
|1074       |9         |66       |159165.18  |Santa Catarina     |
|2981       |9         |70       |126282.40  |Bahia              |
|28         |9         |76       |29114.45   |Rio Grande do Sul  |
|3090       |9         |93       |95697.30   |Minas Gerais       |
|700        |9         |74       |142389.09  |Goiás              |
|2551       |9         |88       |187513.66  |Santa Catarina     |
|1752       |8         |80       |97028.45   |Tocantins          |
|1787       |8         |79       |121494.67  |Ceará           

3) Quais são os usuários que não fizeram nenhuma compra?

In [156]:
q("""
    SELECT x.cod_usuario, y.quantidade
    FROM usuarios x 
    left join vendas y
    on x.cod_usuario =  y.cod_usuario
    where y.quantidade is null and x.cod_usuario is not null
    group by x.cod_usuario, y.quantidade;
""")

+-----------+----------+
|cod_usuario|quantidade|
+-----------+----------+
|19984      |null      |
|25517      |null      |
|41409      |null      |
|68090      |null      |
|87120      |null      |
|195413     |null      |
|200379     |null      |
|219523     |null      |
|246097     |null      |
|362827     |null      |
|371765     |null      |
|417190     |null      |
|485130     |null      |
|91785      |null      |
|67782      |null      |
|90817      |null      |
|18654      |null      |
|76143      |null      |
|54264      |null      |
|69048      |null      |
|63155      |null      |
|18800      |null      |
|22521      |null      |
|5614       |null      |
|22188      |null      |
|50732      |null      |
|149257     |null      |
|206380     |null      |
|221801     |null      |
|224325     |null      |
+-----------+----------+
only showing top 30 rows



4) Qual é o ticket médio (média de valor gasto) e o número total de usuários que fizeram pelo menos uma compra por faixa etária?

In [157]:
q("""
    SELECT count(x.cod_usuario) total_users, 
    cast(avg(y.valor) AS decimal(10,2)) valor_medio,
    x.faixa_etaria
    FROM usuarios x 
    left join vendas y
    on x.cod_usuario =  y.cod_usuario
    where y.quantidade is not null
    group by x.faixa_etaria;
""")

+-----------+-----------+------------------+
|total_users|valor_medio|faixa_etaria      |
+-----------+-----------+------------------+
|64         |12203.02   |Entre 22 a 27 anos|
|43         |8890.13    |Entre 10 a 15 anos|
|50         |12725.95   |Entre 16 a 21 anos|
|80         |10849.48   |Entre 50 a 61 anos|
|42         |16727.19   |Entre 37 a 49 anos|
|39         |10012.63   |Entre 62 a 70 anos|
|56         |8233.04    |Entre 28 a 36 anos|
|29         |11303.10   |Mais de 70 anos   |
+-----------+-----------+------------------+



In [158]:
spark.stop()

# PySpark

**setup**:

In [159]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

mkdir: cannot create directory ‘bases_teste’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1039  100  1039    0     0  16492      0 --:--:-- --:--:-- --:--:-- 16492
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533k  100  533k    0     0  5339k      0 --:--:-- --:--:-- --:--:-- 5339k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0  1637k      0 --:--:-- --:--:-- --:--:-- 1637k


In [160]:
# Setup Spark Session

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round, row_number, max, min, avg
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()


In [161]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

Responda às perguntas a seguir utilizando **Spark DATAFRAMES**.

1) Qual foi o total de compras realizadas, o total de itens comprados e a receita total obtida em todo o período analisado?

In [162]:
vendas.agg(count("quantidade").alias("total compras"), \
           sum("quantidade").alias("total itens") , \
           sum("valor").alias("total receita")).show()

+-------------+-----------+--------------------+
|total compras|total itens|       total receita|
+-------------+-----------+--------------------+
|        20000|     209149|2.1584900165999958E8|
+-------------+-----------+--------------------+



2) Quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga?

In [163]:
ES = ["Paraná", "Rio Grande do Sul", "Santa Catarina", "Espírito Santo", "Minas Gerais", "Rio de Janeiro", "São Paulo"]

vendXuser = vendas.join(usuarios, vendas.cod_usuario == usuarios.cod_usuario).filter(usuarios.estado.isin(ES))

vendXuser.groupBy("estado", "cod_produto")\
  .agg(count("cod_produto").alias("qtd_comprados"),\
    sum("valor").cast("decimal(10,2)").alias("valor_total"),\
    avg("valor").cast("decimal(10,2)").alias("valor_medio")).orderBy(col("qtd_comprados").desc()).show()

+-----------------+-----------+-------------+-----------+-----------+
|           estado|cod_produto|qtd_comprados|valor_total|valor_medio|
+-----------------+-----------+-------------+-----------+-----------+
|     Minas Gerais|         15|           13|     823.35|      63.33|
|     Minas Gerais|          1|            7|  325953.90|   46564.84|
|   Santa Catarina|         17|            6|   30957.30|    5159.55|
|     Minas Gerais|          5|            6|   34086.00|    5681.00|
|        São Paulo|         15|            6|     309.38|      51.56|
|Rio Grande do Sul|         15|            6|     214.57|      35.76|
|     Minas Gerais|         14|            5|    9173.34|    1834.67|
|   Santa Catarina|         13|            4|     346.50|      86.63|
|     Minas Gerais|          9|            4|   17509.50|    4377.38|
|        São Paulo|         13|            4|     405.90|     101.48|
|Rio Grande do Sul|          8|            4|    3513.12|     878.28|
|        São Paulo| 

3) Para cada produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago por eles?

In [167]:
vendas.groupBy("cod_produto").agg(count(vendas.cod_usuario).alias("qtd_usuarios"), max("valor").alias("valor maximo"), min("valor").alias("valor minimo")).orderBy(vendas.cod_produto).show(30)

+-----------+------------+------------+------------+
|cod_produto|qtd_usuarios|valor maximo|valor minimo|
+-----------+------------+------------+------------+
|          1|         942|     91818.0|      4590.9|
|          2|         884|     26915.6|     1345.78|
|          3|         941|     40990.0|      2049.5|
|          4|         964|      1178.0|        58.9|
|          5|         973|      8740.0|       437.0|
|          6|         977|     17998.0|       899.9|
|          7|         985|     26009.8|     1300.49|
|          8|         953|      4391.4|      219.57|
|          9|         982|      7782.0|       389.1|
|         10|         955|     13998.0|       699.9|
|         11|         964|       998.0|        49.9|
|         12|         893|      1550.0|        77.5|
|         13|         928|       198.0|         9.9|
|         14|         925|      2779.8|      138.99|
|         15|         976|        99.8|        4.99|
|         16|         934|       100.0|       

4) Aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado.

In [165]:
vendXprod = vendas.join(produtos, vendas.cod_produto == produtos.cod_produto, "left")

vendXprod.select("valor", "categoria_produto","cod_usuario", "quantidade") \
  .groupBy("categoria_produto","cod_usuario")\
  .agg(count("quantidade").alias("compras"),\
       sum("valor").cast("decimal(10,2)").alias("valor"),\
       (sum("valor") - (sum("valor") / 100) * 10).cast("decimal(10,2)").alias("desconto"))\
  .orderBy("cod_usuario").where(col("compras") > 3).show(10)

+-------------------+-----------+-------+---------+---------+
|  categoria_produto|cod_usuario|compras|    valor| desconto|
+-------------------+-----------+-------+---------+---------+
|         Tecnologia|          3|      4| 94031.30| 84628.17|
|   Eletrodomesticos|          6|      4| 13599.09| 12239.18|
|         Tecnologia|         16|      5|173634.20|156270.78|
|Produtos de limpeza|         17|      5|   508.52|   457.67|
|   Casa e bem-estar|         23|      4| 27189.31| 24470.38|
|   Casa e bem-estar|         34|      4| 31671.05| 28503.95|
|   Casa e bem-estar|         37|      4| 17668.28| 15901.45|
|         Tecnologia|         39|      4|129795.30|116815.77|
|Produtos de limpeza|         41|      4|   260.71|   234.64|
|   Casa e bem-estar|         87|      4| 39324.70| 35392.23|
+-------------------+-----------+-------+---------+---------+
only showing top 10 rows



# FIM!